# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [145]:
import numpy as np
import pandas as pd 
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter


## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [146]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [147]:

symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2330058746581,
 'week52high': 139.35,
 'week52low': 57.62,
 'week52change': 0.8590321044779489,
 'sharesOutstanding': 17038198938,
 'float': 0,
 'avg10Volume': 116708803,
 'avg30Volume': 111651344,
 'day200MovingAvg': 112.56,
 'day50MovingAvg': 124.98,
 'employees': 0,
 'ttmEPS': 3.42,
 'ttmDividendRate': 0.8435708575786005,
 'dividendYield': 0.006128093878643422,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-05',
 'nextEarningsDate': '0',
 'peRatio': 40.3710696261638,
 'beta': 1.158741819693695,
 'maxChangePercent': 51.02421460839057,
 'year5ChangePercent': 4.430014230368664,
 'year2ChangePercent': 2.5480828695531,
 'year1ChangePercent': 0.8991629711891141,
 'ytdChangePercent': 0.888892853604598,
 'month6ChangePercent': 0.5161777424446679,
 'month3ChangePercent': 0.19191593567173898,
 'month1ChangePercent': 0.162284872320447,
 'day30ChangePercent': 0.164334684163078,
 'day5ChangePercent': 0.02307171858854497}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [148]:
data['year1ChangePercent'] 

0.8991629711891141

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [149]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [150]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
          final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                    ],
                index = my_columns),
                    ignore_index = True
          )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,119.089,0.401905,N/A
1,AAL,16.236,-0.444209,N/A
2,AAP,165.870,-0.00281026,N/A
3,AAPL,139.940,0.892498,N/A
4,ABBV,110.790,0.240477,N/A
...,...,...,...,...
500,YUM,114.440,0.096189,N/A
501,ZBH,158.490,0.0247136,N/A
502,ZBRA,391.780,0.500303,N/A
503,ZION,44.780,-0.139028,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [151]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.310,2.1241,N/A
1,345,NVDA,529.750,1.2112,N/A
2,387,PYPL,237.850,1.11212,N/A
3,23,ALB,150.328,1.1084,N/A
4,275,LB,38.626,1.10392,N/A
5,31,AMD,93.817,0.965838,N/A
6,85,CDNS,138.180,0.951182,N/A
7,179,FCX,27.690,0.946166,N/A
8,339,NOW,562.524,0.910481,N/A
9,6,ABMD,321.910,0.899324,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [152]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')

    try:
        float(portfolio_size)
    except ValueError:
        print("That's not a number! \nPlease try again!")
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

10000000


In [153]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,78,CARR,38.310,2.1241,5220
1,345,NVDA,529.750,1.2112,377
2,387,PYPL,237.850,1.11212,840
3,23,ALB,150.328,1.1084,1330
4,275,LB,38.626,1.10392,5177
5,31,AMD,93.817,0.965838,2131
6,85,CDNS,138.180,0.951182,1447
7,179,FCX,27.690,0.946166,7222
8,339,NOW,562.524,0.910481,355
9,6,ABMD,321.910,0.899324,621


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [274]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns),
                ignore_index = True
        )
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118,N/A,0.386493,N/A,0.35936,N/A,0.17663,N/A,0.0278899,N/A,N/A
1,AAL,16.167,N/A,-0.449376,N/A,0.196301,N/A,0.303051,N/A,0.0589072,N/A,N/A
2,AAP,164.859,N/A,-0.0028561,N/A,0.104182,N/A,0.0349952,N/A,0.0485222,N/A,N/A
3,AAPL,134.52,N/A,0.907161,N/A,0.503427,N/A,0.186199,N/A,0.158831,N/A,N/A
4,ABBV,110.62,N/A,0.247117,N/A,0.118197,N/A,0.223952,N/A,-0.00186859,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,114.7,N/A,0.0977858,N/A,0.280436,N/A,0.215091,N/A,0.0218148,N/A,N/A
501,ZBH,152.8,N/A,0.0250371,N/A,0.309273,N/A,0.147929,N/A,0.0180162,N/A,N/A
502,ZBRA,394.15,N/A,0.483544,N/A,0.518375,N/A,0.473327,N/A,0.00280288,N/A,N/A
503,ZION,43.62,N/A,-0.142267,N/A,0.324901,N/A,0.506263,N/A,0.0873257,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [275]:
hqm_dataframe.replace(to_replace=[None], value=0, inplace=True)

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month',
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.000,N/A,0.386493,0.859406,0.359360,0.70297,0.176630,0.534653,0.027890,0.647525,N/A
1,AAL,16.167,N/A,-0.449376,0.0277228,0.196301,0.419802,0.303051,0.790099,0.058907,0.80198,N/A
2,AAP,164.859,N/A,-0.002856,0.378218,0.104182,0.249505,0.034995,0.217822,0.048522,0.752475,N/A
3,AAPL,134.520,N/A,0.907161,0.978218,0.503427,0.877228,0.186199,0.554455,0.158831,0.976238,N/A
4,ABBV,110.620,N/A,0.247117,0.738614,0.118197,0.271287,0.223952,0.645545,-0.001869,0.4,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,114.700,N/A,0.097786,0.530693,0.280436,0.564356,0.215091,0.621782,0.021815,0.607921,N/A
501,ZBH,152.800,N/A,0.025037,0.429703,0.309273,0.619802,0.147929,0.467327,0.018016,0.590099,N/A
502,ZBRA,394.150,N/A,0.483544,0.918812,0.518375,0.885149,0.473327,0.934653,0.002803,0.465347,N/A
503,ZION,43.620,N/A,-0.142267,0.194059,0.324901,0.645545,0.506263,0.950495,0.087326,0.883168,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [276]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,118.000,N/A,0.386493,0.859406,0.359360,0.70297,0.176630,0.534653,0.027890,0.647525,0.686139
1,AAL,16.167,N/A,-0.449376,0.0277228,0.196301,0.419802,0.303051,0.790099,0.058907,0.80198,0.509901
2,AAP,164.859,N/A,-0.002856,0.378218,0.104182,0.249505,0.034995,0.217822,0.048522,0.752475,0.399505
3,AAPL,134.520,N/A,0.907161,0.978218,0.503427,0.877228,0.186199,0.554455,0.158831,0.976238,0.846535
4,ABBV,110.620,N/A,0.247117,0.738614,0.118197,0.271287,0.223952,0.645545,-0.001869,0.4,0.513861
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,114.700,N/A,0.097786,0.530693,0.280436,0.564356,0.215091,0.621782,0.021815,0.607921,0.581188
501,ZBH,152.800,N/A,0.025037,0.429703,0.309273,0.619802,0.147929,0.467327,0.018016,0.590099,0.526733
502,ZBRA,394.150,N/A,0.483544,0.918812,0.518375,0.885149,0.473327,0.934653,0.002803,0.465347,0.80099
503,ZION,43.620,N/A,-0.142267,0.194059,0.324901,0.645545,0.506263,0.950495,0.087326,0.883168,0.668317


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [277]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace= True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,532.400,N/A,0.908057,0.980198,0.976207,0.992079,0.641675,0.988119,0.097502,0.918812,0.969802
1,ALB,150.900,N/A,1.097795,0.992079,0.951207,0.986139,0.699100,0.990099,0.066267,0.825743,0.948515
2,FCX,27.709,N/A,0.941872,0.988119,1.290915,0.99802,0.623930,0.980198,0.052844,0.772277,0.934653
3,SIVB,388.270,N/A,0.523563,0.922772,0.852989,0.984158,0.608605,0.970297,0.063342,0.815842,0.923267
4,PAYC,456.600,N/A,0.670630,0.952475,0.497455,0.869307,0.479359,0.936634,0.074338,0.849505,0.90198
5,ALXN,156.900,N/A,0.449434,0.90495,0.404303,0.758416,0.396664,0.89505,0.259206,0.99802,0.889109
6,TWTR,56.043,N/A,0.696865,0.956436,0.849456,0.982178,0.217288,0.627723,0.174011,0.982178,0.887129
7,DIS,185.810,N/A,0.225845,0.70495,0.613581,0.938614,0.420776,0.908911,0.207463,0.992079,0.886139
8,IDXX,507.000,N/A,0.882833,0.974257,0.521756,0.891089,0.297005,0.786139,0.091341,0.891089,0.885644
9,CDNS,136.198,N/A,0.937362,0.986139,0.437506,0.809901,0.272042,0.740594,0.155257,0.970297,0.876733


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [278]:
portfolio_input()

In [279]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALGN,532.400,37,0.908057,0.980198,0.976207,0.992079,0.641675,0.988119,0.097502,0.918812,0.969802
1,ALB,150.900,132,1.097795,0.992079,0.951207,0.986139,0.699100,0.990099,0.066267,0.825743,0.948515
2,FCX,27.709,721,0.941872,0.988119,1.290915,0.99802,0.623930,0.980198,0.052844,0.772277,0.934653
3,SIVB,388.270,51,0.523563,0.922772,0.852989,0.984158,0.608605,0.970297,0.063342,0.815842,0.923267
4,PAYC,456.600,43,0.670630,0.952475,0.497455,0.869307,0.479359,0.936634,0.074338,0.849505,0.90198
5,ALXN,156.900,127,0.449434,0.90495,0.404303,0.758416,0.396664,0.89505,0.259206,0.99802,0.889109
6,TWTR,56.043,356,0.696865,0.956436,0.849456,0.982178,0.217288,0.627723,0.174011,0.982178,0.887129
7,DIS,185.810,107,0.225845,0.70495,0.613581,0.938614,0.420776,0.908911,0.207463,0.992079,0.886139
8,IDXX,507.000,39,0.882833,0.974257,0.521756,0.891089,0.297005,0.786139,0.091341,0.891089,0.885644
9,CDNS,136.198,146,0.937362,0.986139,0.437506,0.809901,0.272042,0.740594,0.155257,0.970297,0.876733


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [280]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index= False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [281]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [282]:
column_formats ={
    
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile', percent_template],
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template],
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template],
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Retrun Percentile', percent_template],
    'L':['HQM Score', percent_template]
}
10
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [283]:
writer.save()